# Install Packages
**This code predicts spatially the Abolute Dynamic Topography of the Baltic Sea using integration of autoregressive-physical model (meteorological features such as SST, Uwind, Vwind, SST) and DOY based on a MultiVariate CNN Deep Learning algorithm.**

#### import packages

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os

#### make a time window

In [2]:
def window_data(df, window, feature_col_number, target_col_number):
    """
    This creates sliding windows of data from a DataFrame (df). It extracts a sequence of features and a corresponding target value for each window.
    The extracted features and targets are stored in separate lists (X and y). The function then returns these lists as NumPy arrays for further processing.
    """
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i : (i + window), feature_col_number].values
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).astype(np.float64).reshape(-1, 1)

In [4]:
df = pd.read_csv('Total.csv', infer_datetime_format=True,skiprows=1)
df.dropna(inplace=True)
df

,DOY,SLP,Uwind,Vwind,SST,SSH
8,1,1.2715,9.2795,1.958700,5.4714,0.31793
9,1,1.2711,7.9834,1.407400,5.5952,0.38918
61,1,1.2715,9.4015,2.011700,5.5138,0.32000
62,1,1.2710,8.0667,1.473900,5.6538,0.39404
113,1,1.2716,9.7303,2.250300,5.7502,0.32646
...,...,...,...,...,...,...
3945984,365,1.2665,-2.7609,0.013072,3.3476,0.69167
3945985,365,1.2663,-2.2076,0.270130,3.4690,0.70275
3946037,365,1.2664,-2.6938,-0.126070,3.2216,0.68988
3946038,365,1.2660,-2.1865,0.172910,3.4000,0.70851


In [5]:
df = df[['SSH','SST','SLP','Uwind' , 'Vwind','DOY']]
df.columns = ['DT', 'SST', 'SLP','Uwind','Vwind','DOY']
df.head()

,DT,SST,SLP,Uwind,Vwind,DOY
8,0.31793,5.4714,1.2715,9.2795,1.9587,1
9,0.38918,5.5952,1.2711,7.9834,1.4074,1
61,0.32000,5.5138,1.2715,9.4015,2.0117,1
62,0.39404,5.6538,1.2710,8.0667,1.4739,1
113,0.32646,5.7502,1.2716,9.7303,2.2503,1


In [6]:
# It imports the StandardScaler class from the sklearn.preprocessing module and creates an instance of it called scaler.
# The variable window_size is set to 7, representing the size of the sliding window used for data extraction.
# The window_data function is called multiple times to extract specific features and targets from the DataFrame df. The extracted data is stored in variables such as SST, SLP, Uwind, Vwind, and DOY.
# The extracted data is then standardized using the fit_transform method of the scaler object, and the standardized data is reassigned to the same variables (SST, SLP, Uwind, Vwind, DOY).
# The data is split into training and testing sets based on a specified split point (80% for training, 20% for testing). Separate arrays are created for training and testing data for features (X_train, X_test) and targets (y_train, y_test).
# Similarly, separate arrays are created for the other extracted features (SST_train, SLP_train, Uwind_train, Vwind_train, DOY_train).
# The training and testing feature arrays are reshaped to have a third dimension of size 1. This reshaping is necessary for certain types of neural network models.
# The reshaped arrays for each feature are concatenated along the third dimension using the np.concatenate function. The resulting array (data_train) contains the training data with all the features combined.
# The same reshaping process is applied to the testing feature arrays, and separate arrays are created for each feature (SST_test, SLP_test, Uwind_test, Vwind_test, DOY_test).

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

window_size = 7 

(X, y) = window_data(df, window_size, 0, 0)

(SST, _) = window_data(df, window_size, 1, 1)
(SLP, _) = window_data(df, window_size, 2, 1)
(Uwind, _) = window_data(df, window_size, 3, 1)
(Vwind, _) = window_data(df, window_size, 4, 1)
(DOY, _) = window_data(df, window_size, 5, 1)


X = scaler.fit_transform(X)


SST = scaler.fit_transform(SST)
SLP = scaler.fit_transform(SLP)
Uwind = scaler.fit_transform(Uwind)
Vwind = scaler.fit_transform(Vwind)
DOY = scaler.fit_transform(DOY)


split = int(0.8 * len(X))
X_train = X[: split - 1]
X_test = X[split:]

y_train = y[: split - 1]
y_test = y[split:]

SST_train = SST[: split - 1]
SLP_train = SLP[: split - 1]
Uwind_train = Uwind[: split - 1]
Vwind_train = Vwind[: split - 1]
DOY_train = DOY[: split - 1]


SST_test = SST[split:]
SLP_test = SLP[split:]
Uwind_test = Uwind[split:]
Vwind_test = Vwind[split:]
DOY_test = DOY[split:]


X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
SST_train = SST_train.reshape((SST_train.shape[0], SST_train.shape[1], 1))
SLP_train = SLP_train.reshape((SLP_train.shape[0], SLP_train.shape[1], 1))
Uwind_train = Uwind_train.reshape((Uwind_train.shape[0], Uwind_train.shape[1], 1))
Vwind_train = Vwind_train.reshape((Vwind_train.shape[0], Vwind_train.shape[1], 1))
DOY_train = DOY_train.reshape((DOY_train.shape[0], DOY_train.shape[1], 1))
data_train = np.concatenate((X_train,
                             SST_train, SLP_train, Uwind_train, Vwind_train
                             , DOY_train
                            ), axis=2)


X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
SST_test = SST_test.reshape((SST_test.shape[0], SST_test.shape[1], 1))
SLP_test = SLP_test.reshape((SLP_test.shape[0], SLP_test.shape[1], 1))
Uwind_test = Uwind_test.reshape((Uwind_test.shape[0], Uwind_test.shape[1], 1))
Vwind_test = Vwind_test.reshape((Vwind_test.shape[0], Vwind_test.shape[1], 1))
DOY_test = DOY_test.reshape((DOY_test.shape[0], DOY_test.shape[1], 1))

In [7]:
data_test = np.concatenate((X_test,
                            SST_test, SLP_test, Uwind_test, Vwind_test
                            ,DOY_test
                           ), axis=2)
# X_train.shape

# Conv2D

In [17]:
def basic_conv2D(n_filters=10, fsize=5, window_size=5, n_features=2):
    model = keras.Sequential()
    model.add(tf.keras.layers.Conv2D(n_filters, (1,fsize), padding="same", activation="relu", input_shape=(window_size, n_features, 1)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1000, activation='relu'))
    model.add(tf.keras.layers.Dense(100))
    model.add(tf.keras.layers.Dense(1))
    model.compile(optimizer="adam", loss="mean_squared_error")    
    return model

In [13]:
data_train_wide = data_train.reshape((data_train.shape[0], data_train.shape[1], data_train.shape[2], 1))
data_test_wide = data_test.reshape((data_test.shape[0], data_test.shape[1], data_test.shape[2], 1))
data_train_wide.shape
# 2dcnn = basic_conv2D(n_filters=24, fsize=2, window_size=window_size, n_features=data_train_wide.shape[2])

(640348, 7, 6, 1)

In [11]:
# load saved model
from keras.models import load_model
conv2d = load_model('ST_Conv2d.h5')
conv2d.summary()
# hist_conv2d = conv2d.fit(data_train_wide, y_train, epochs=10)
# plt.plot(hist_conv2d.history["loss"])
# plt.title("Conv2D")
# plt.legend(["training loss"])
# plt.show()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 7, 6, 24)          72        
                                                                 
 flatten (Flatten)           (None, 1008)              0         
                                                                 
 dense (Dense)               (None, 1000)              1009000   
                                                                 
 dense_1 (Dense)             (None, 100)               100100    
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 1,109,273
Trainable params: 1,109,273
Non-trainable params: 0
_________________________________________________________________


In [36]:
conv2d.evaluate(data_test_wide, y_test, verbose=0)

from sklearn.metrics import r2_score
predictions = conv2d.predict(data_test_wide)
test_r2 = r2_score(y_test, predictions)
test_r2

5003/5003 [==============================] - 11s 2ms/step


0.9105584312559938

In [37]:
conv2d_df = pd.DataFrame()
conv2d_df['Predict'] = predictions[:,0]
conv2d_df['Actual'] = y_test[:,0]
# conv2d_df.head()

,Actual,Predict
0,0.21816,0.474035
1,0.25698,0.246772
2,0.20429,0.258457
3,0.26036,0.224847
4,0.15674,0.272733
5,0.16680,0.227830
6,0.26227,0.232267
7,0.18126,0.261867
8,0.19178,0.199052
9,0.16398,0.199012
